공간 영역 처리 : 커널(마스크, 윈도우)을 이용하여 회선을 수행하여 처리가 이루어짐


### 7.1 회선
영역 의미1 : 두 개의 다른 범위의 구분 (공간 영역, 주파수 영역)

- 공간 영역 : 영상들이 다루어지는 화소 공간을 의미

영역 의미2 : 영역 기반 처리 표현에서 사용하는 영역 -> 화소가 모인 특정 범위의 화소 배열

- 화소 기반 처리 : 화소값 각각에 대해 연산을 수행
- 영영 기반 처리 : 마스크를 기반으로 연산을 수행

마스크 기반 처리 : 마스크 내의 원소 값과 공간 영영에 있는 화소 값을 대응되게 곱하여 출력 화소 값을 계산
- 마스크를 커널, 윈도우, 필터 등으로 부름

회선 : 마스크 기반 처리를 모든 화소값에 대해 이동하면서 수행하는 것 


##### 7.1.2 블러링
화소 값이 급격하게 변화하는 것을 점진적으로 변하게 함

커널 마스크를 이용하여 회선을 수행함 -> 마스크 전체 원소의 합이 1이 되어야 영상을 밝기가 유지됨 (주변 값의 합 / 주변 값의 개수)

In [4]:
# 7.1.1 회선이용 블러링
import numpy as np, cv2

# 회선 수행 함수 - 행렬 처리 방식
def filter(image, mask):
    print("image.shape", image.shape)
    rows, cols = image.shape[:2]
    print("rows",rows)
    print("cols",cols)
    dst = np.zeros((rows, cols), np.float32) # 회선 결과를 저장할 행렬
    ycenter, xcenter = rows//2, cols//2 # 마스크 중심 좌표

    print(ycenter)
    print(rows)
    ################## 에러
    for i in range(ycenter, rows - ycenter): # 회선
        for j in range(xcenter, cols - xcenter):
            y1, y2 = i - ycenter, i + ycenter + 1 # 관심 영역 높이 범위
            x1, x2 = j - xcenter, j + xcenter + 1 # 관심 영역 너비 범위
            roi = image[y1:y2, x1:x2].astype('float32') # 관심 영역 형변환
            tmp = cv2.multiply(roi, mask) # 회선 적용 - 원소간 곱셈
            dst[i, j] = cv2.sumElems(tmp)[0] # 자로형 변환
            print(dst[i, j])
    return dst

# 회선 수행 함수 - 화소 직접 근접
def filter2(image, mask):
    rows, cols = image.shape[:2]
    dst = np.zeros((rows, cols), np.float32) # 회선 결과를 저장할 행렬
    ycenter, xcenter = rows//2, cols//2 # 마스크 중심 좌표

    for i in range(ycenter, rows - ycenter): # 회선
        for j in range(xcenter, cols - xcenter):
            sum = 0.0
            for u in range(mask.shape[0]):
                for v in range(mask.shape[1]):
                    y, x = i + u - ycenter, j + v - xcenter
                    sum += image[y, x] * mask[u, v] # 회선 수식
            dst[i, j] = sum
    return dst

image = cv2.imread("images_07/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

data = [1/9, 1/9, 1/9, 1/9, 1/9, 1/9, 1/9, 1/9, 1/9]
mask = np.array(data, np.float32).reshape(3,3)

blur1 = filter(image, mask)
blur2 = filter2(image, mask)

# blur1 = blur1.astype('uint8') # 정수형으로 변환 방법1
blur1 = cv2.convertScaleAbs(blur1) # 정수형으로 변환 방법2
blur2 = cv2.convertScaleAbs(blur2) # 정수형으로 변환 방법2


cv2.imshow("image", image)
cv2.imshow("blur1", blur1)
cv2.imshow("blur2", blur2)
cv2.waitKey(0)



image.shape (300, 400)
rows 300
cols 400
150
300


-1

##### 7.1.3 샤프닝
이웃 화소끼리 차이를 크게 되도록 출력화소를 만들어 날카로운 느낌이 나도록 하는 것

중심계수는 1보다 크게 구성하고, 주변 화소는 음수 값을 갖게 하여 비중을 감소

In [1]:
# 7.1.2 회선이용 샤프닝
import numpy as np, cv2
from Common.filters import filter

image = cv2.imread("images_07/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

data1 = [0, -1 , 0,
         -1, 5, -1,
         0, -1, 0]

data2 = [-1, -1 , -1,
         -1, 9, -1,
         -1, -1, -1]

mask1 = np.array(data1, np.float32).reshape(3,3)
mask2 = np.array(data2, np.float32).reshape(3,3)

sharpen1 = filter(image, mask1) # 회선 수행
sharpen2 = filter(image, mask2)
sharpen1 = cv2.convertScaleAbs(sharpen1) # 형변환
sharpen2 = cv2.convertScaleAbs(sharpen2)

cv2.imshow("image", image)
cv2.imshow("sharpen1", sharpen1)
cv2.imshow("sharpen2", sharpen2)
cv2.waitKey(0)


ImportError: cannot import name 'filter' from 'Common.filters' (d:\github\image-processing\ch07\Common\filters.py)